In [ ]:
#@title 환경 구성

%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
# !git clone https://github.com/Beomi/ko-lm-evaluation-harness.git
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

0.    "unsloth/llama-2-7b",
1.    "unsloth/llama-3-8b-bnb-4bit",
2.    "unsloth/llama-3-8b-Instruct-bnb-4bit",

In [ ]:
#@title Config 설정 & Model Selection

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/llama-2-7b",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model_id = 0 #@param {type: "integer"}
CoT적용 = False #@param {type: "boolean"}
num_dataset = 1000 #@param {type: "integer"}
repo_name_map = {
    0: {
        True: f'llama-2-7b-unsloth-KoCoT-{num_dataset}-with-CoT',
        False: f'llama-2-7b-unsloth-KoCoT-{num_dataset}'
    },
    1: {
        True: f'llama-3-8b-unsloth-KoCoT-{num_dataset}-with-CoT',
        False: f'llama-3-8b-unsloth-KoCoT-{num_dataset}'
    }
}

REPO_NAME = repo_name_map[model_id][CoT적용]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = fourbit_models[model_id],
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
#@title LoRA adapter 적용

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
#@title Dataset Preprocessing

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

if CoT적용:

  def formatting_prompts_func(examples):
      inputs = examples["source"]
      outputs = examples["rationale"]
      texts = []
      for input, output in zip(inputs, outputs):
          # Must add EOS_TOKEN, otherwise your generation will go on forever!
          text = alpaca_prompt.format(input, output) + EOS_TOKEN
          texts.append(text)
      return { "text" : texts, }

else:

  def formatting_prompts_func(examples):
      inputs = examples["source"]
      outputs = examples["target"]
      texts = []
      for input, output in zip(inputs, outputs):
          # Must add EOS_TOKEN, otherwise your generation will go on forever!
          text = alpaca_prompt.format(input, output) + EOS_TOKEN
          texts.append(text)
      return { "text" : texts, }

from datasets import load_dataset
dataset_name = "kyujinpy/KoCoT_2000" #@param {type: "string"}
dataset = load_dataset(dataset_name, split=f'train[:{num_dataset}]')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.168 GB.
3.824 GB of memory reserved.


In [ ]:
#@title Training

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
1,1.464500
2,1.295100
3,1.463100
4,1.442900
5,1.390700
6,1.263100
7,1.199600
8,1.228000
9,1.170000
10,1.096300


In [ ]:
#@title save on hf repo

from huggingface_hub import notebook_login
from getpass import getpass

notebook_login()

AUTH_TOKEN = getpass("Token:")

## Upload to Huggingface Hub(16bit)
# model.push_to_hub_merged(REPO_NAME, tokenizer, save_method = "merged_16bit", token = AUTH_TOKEN)

## Upload to Huggingface Hub(adapter)
model.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)
tokenizer.push_to_hub(
    REPO_NAME,
    use_temp_dir=True,
    use_auth_token=AUTH_TOKEN
)

Token:··········


README.md:   0%|          | 0.00/572 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

Saved model to https://huggingface.co/llama-2-7b-unsloth-KoCoT-1000


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

In [ ]:
#@title load model from hf repo
# from transformers import AutoModelForCausalLM

# model_ = AutoModelForCausalLM.from_pretrained("swoos/llama_unsloth_test")

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

282.5855 seconds used for training.
4.71 minutes used for training.
Peak reserved memory = 17.352 GB.
Peak reserved memory for training = 11.758 GB.
Peak reserved memory % of max memory = 78.275 %.
Peak reserved memory for training % of max memory = 53.04 %.


# 실험중

In [ ]:
%%capture
!git clone https://github.com/Beomi/ko-lm-evaluation-harness.git

In [ ]:
cd ko-lm-evaluation-harness

/content/ko-lm-evaluation-harness


In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0"
!pip install -r requirements.txt
!pip install -U datasets trl accelerate peft bitsandbytes transformers trl huggingface_hub

In [ ]:
!python main.py \
--model hf-causal-experimental \
--model_args pretrained=swoos/llama-3-8b-unsloth-KoCoT-1000-with-CoT,use_accelerate=true,trust_remote_code=true \
--tasks kobest_boolq,kobest_hellaswag,kobest_copa,kobest_sentineg \
--num_fewshot 5 \
--no_cache \
--limit 500 \
--batch_size 4 \
--output_path results/all/llama-3-8b-unsloth-KoCoT-1000-with-CoT/llama3_lima_5_shot_1000_CoT.json

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
질문: 다음 문장이 내용에 있는 사실과 일치하는가?
문장: 히말라야식 요리에는 감자가 많이 들어가는가?
응답:  예

내용: 아르헨티나의 본격적인 문학은 독립 후의 혼란기에 낭만주의가 대두함으로써 시작되었다. 탄압 하의 부에노스아이레스를 묘사한 소설 <아말리아>로 유명한 호세 마르몰(1817-1871), 문화사의 고전 <파쿤도, 문명이냐 야만이냐>의 저자인 도밍고 파스티노 사르미엔토(1811-1888)도 있으나, 이 시기의 문학을 대표하는 것은 누가 무어라 해도 가우초의 시로 초원에 사는 소치기의 생활을 낭만적으로 노래한 것이다. 
질문: 다음 문장이 내용에 있는 사실과 일치하는가?
문장: 낭만주의는 아르헨티나의 독립 전에 대두하였는가?
응답: 
내용: 마리안은 밤중에 복도를 거닐면서 하얀 웨딩드레스를 입은 엘로이즈의 환상을 보기 시작한다. 이윽고 둘은 산책을 하면서 동굴 속에서 처음으로 키스를 나누고, 그날 밤 소피가 잠자리에 들면 열정적으로 사랑을 나누었다. 
질문: 다음 문장이 내용에 있는 사실과 일치하는가?
문장: 엘로이즈와 마리안은 시간이 흐름에 따라, 사랑에 빠졌는가?
응답:  예

내용: 〈목걸이〉는 프랑스의 소설가 모파상이 지어 1884년에 발표한 단편소설이다. 교훈적, 비판적, 묘사적 성격을 띄는 자연주의 소설로 극적 반전의 묘미를 느낄 수 있는 작품이다. 한 젊은 여인의 사치스럽고 우아한 귀족 생활을 동경하는 욕심때문에 고달픈 삶을 살게 된다는 운명의 아이러니를 주제로 다룬 작품으로 1884년 프랑스의 작가 기 드 모파상에 의해 완성된 단편 소설이다.
질문: 다음 문장이 내용에 있는 사실과 일치하는가?
문장: 〈목걸이〉 속의 젊은 여인은 귀족 생활을 동경하는가?
응답:  예

내용: 《센과 치히로의 행방불명》(일본어: 千せんと千尋ちひろの神隠かみかくし 센토 치히로노 가미카쿠시[*], 헵번 로마자: Sen to Chihiro no kamikakushi, 영어: Spirited Awa

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model_) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "" # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model_.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

NameError: name 'FastLanguageModel' is not defined

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "swoos/llama-3-8b-unsloth-KoCoT-1000-final-adapter", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

KeyboardInterrupt: 

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>